In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

!conda install -c conda-forge beautifulsoup4 --yes
from bs4 import BeautifulSoup

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.6.14
  latest version: 4.7.5

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /srv/conda/envs/notebook

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.2 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.49-py_0
  geopy              conda-forge/noarch::geopy-1.20.0-py_0

T

In [2]:
#csv from https://simplemaps.com/data/nl-cities
Netherlands = pd.read_csv('nl.csv')
Netherlands.sort_values(by = 'population', ascending = False)
Netherlands.drop(columns = ['country', 'iso2', 'admin', 'capital'], inplace = True)
Netherlands.head()

,city,lat,lng,population,population_proper
0,The Hague,52.083333,4.300000,1406000.0,501725.0
1,Amsterdam,52.350000,4.916667,1031000.0,741636.0
2,Rotterdam,51.916667,4.500000,1005000.0,598199.0
3,Utrecht,52.093813,5.119095,640000.0,316448.0
4,Eindhoven,51.450000,5.466667,398053.0,209620.0


In [3]:
Netherlands = Netherlands.rename(columns = {'lat':'Latitude', 'lng': 'Longitude', 'city': 'City'})
Netherlands.head()

,City,Latitude,Longitude,population,population_proper
0,The Hague,52.083333,4.300000,1406000.0,501725.0
1,Amsterdam,52.350000,4.916667,1031000.0,741636.0
2,Rotterdam,51.916667,4.500000,1005000.0,598199.0
3,Utrecht,52.093813,5.119095,640000.0,316448.0
4,Eindhoven,51.450000,5.466667,398053.0,209620.0


In [4]:
Netherlands_Clean=Netherlands.drop(columns= ['population', 'population_proper'])
Netherlands_Clean.head()

,City,Latitude,Longitude
0,The Hague,52.083333,4.300000
1,Amsterdam,52.350000,4.916667
2,Rotterdam,51.916667,4.500000
3,Utrecht,52.093813,5.119095
4,Eindhoven,51.450000,5.466667


In [5]:
address = 'Netherlands'

geolocator = Nominatim(user_agent="netherlands")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Netherlands are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Netherlands are 52.24726925, 5.54122345487672.


In [6]:
# create map of New York using latitude and longitude values
map_netherlands = folium.Map(location=[latitude, longitude], zoom_start=7)

# add markers to map
for lat, lng, city in zip(Netherlands_Clean['Latitude'], Netherlands_Clean['Longitude'], Netherlands_Clean['City']):
    label = ' {}'.format(city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_netherlands)  
    
map_netherlands

In [11]:
#CLIENT_ID = 'PNNTTIIPRW4E5HOYFVP2JU20KUJUX0YF3A15WYIRHRHZHIF3' # your Foursquare ID
#CLIENT_SECRET = '3H5OSLL0KIWIXIOUVXQX1XJHYIDXZKLJKELUXXJ5PSQQSF4N' # your Foursquare Secret
CLIENT_ID = 'PE2PMUZURJHESEPZGJSBOI3CCY422YUBSEOBY4UVXMN4QYU0' # your Foursquare ID
CLIENT_SECRET = 'XAR5KYUE3MOQF4AFJPBXKT2HGVKG5PGXN2ZKBHJNULSFAFQQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PE2PMUZURJHESEPZGJSBOI3CCY422YUBSEOBY4UVXMN4QYU0
CLIENT_SECRET:XAR5KYUE3MOQF4AFJPBXKT2HGVKG5PGXN2ZKBHJNULSFAFQQ


In [12]:
#Function to apply it to every Postcode we have.
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
netherlands_venues = getNearbyVenues(names=Netherlands_Clean['City'],
                                   latitudes=Netherlands_Clean['Latitude'],
                                   longitudes=Netherlands_Clean['Longitude']
                                  )

The Hague
Amsterdam
Rotterdam
Utrecht
Eindhoven
Haarlemmerliede
Groningen
Arnhem
’s-Hertogenbosch
Leeuwarden
Maastricht
Zwolle
Assen
Middelburg
Halfweg
Franeker
Gieten
Bergeijk
Odijk
Grootegast
Oostzaan
Bleskensgraaf
Haarlem
Lelystad
Valkenswaard
Stein
De Steeg
Diever
Uithuizen
Uithoorn
Bergschenhoek
Zundert
Nunspeet
Nuth
Opheusden
Veenendaal
Oost-Vlieland
Middelharnis
Almelo
Bladel
Hulst
Zwijndrecht
Epe
Hardenberg
Hengelo
Lisse
Nuenen
Vught
Blaricum
Marum
Elburg
Hoogezand
Uden
Hendrik-Ido-Ambacht
Oud-Beijerland
Goor
Urk
Doorn
Denekamp
Heerenveen
Roermond
Waalre
Ballum
Weert
Valkenburg
Etten-Leur
West-Terschelling
Rucphen
Vianen
Best
Meerssen
Kruiningen
Voerendaal
Coevorden
Soest
Landgraaf
Meppel
Geldrop
Beilen
Helmond
Hellevoetsluis
Waddinxveen
Kollum
Borculo
Made
Groesbeek
Beuningen
Werkendam
Zevenbergen
Vlissingen
Boxtel
Boxmeer
Reuver
Mill
Hoogkarspel
Beneden-Leeuwen
Gendringen
Joure
Druten
Schiedam
Sliedrecht
Hazerswoude-Rijndijk
Beverwijk
Sint Annaparochie
Zutphen
Vaals
Breda
Roe

In [14]:
print(netherlands_venues.shape)
netherlands_venues.head()

(4202, 7)


,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Hague,52.083333,4.3,Brood & Koffie bij Clarence,52.082146,4.299441,Sandwich Place
1,The Hague,52.083333,4.3,Coffeeshop Dizzy Duck,52.083740,4.299205,Smoke Shop
2,The Hague,52.083333,4.3,Restaurant Hortus,52.084696,4.299527,Vegetarian / Vegan Restaurant
3,The Hague,52.083333,4.3,Hilton The Hague,52.084430,4.303635,Hotel
4,The Hague,52.083333,4.3,Wicked Wines,52.084369,4.299425,Wine Bar


In [15]:
#Grouping by Neighborhood will be neccesarily, and also more readable.
netherlands_venues.groupby('City').count().head()

,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
City,,,,,,
Aalsmeer,12,12,12,12,12,12
Aalten,8,8,8,8,8,8
Abbenbroek,3,3,3,3,3,3
Alblasserdam,9,9,9,9,9,9
Alkmaar,16,16,16,16,16,16


In [16]:
print('There are {} uniques categories.'.format(len(netherlands_venues['Venue Category'].unique())))

There are 303 uniques categories.


In [17]:
# one hot encoding
netherlands_onehot = pd.get_dummies(netherlands_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
netherlands_onehot['City'] = netherlands_venues['City'] 

# move neighborhood column to the first column
fixed_columns = [netherlands_onehot.columns[-1]] + list(netherlands_onehot.columns[:-1])
netherlands_onehot = netherlands_onehot[fixed_columns]
netherlands_onehot.head()

,Zoo,ATM,Accessories Store,Afghan Restaurant,Airport Lounge,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brasserie,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Camera Store,Campground,Candy Store,Caribbean Restaurant,Casino,Castle,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,City,Clothing Store,Cocktail Bar,Coffee Shop,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Distillery,Dog Run,Donut Shop,Drugstore,Dutch Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fireworks Store,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Friterie,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Field,Hockey Rink,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Hunting Supply,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Island,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Library,Light Rail Station,Lighthouse,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Notary,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Perfume Shop,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pool Hall,Pub,Public Art,Racetrack,Record Shop,Recreation Center,Rental Car Location,Rental Service,Resort,Rest Area,Restaurant,River,Road,Rock Club,Roof Deck,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shawarma Place,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Smoke Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Stadium,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Train Station,Tram Station,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Resta

In [18]:
netherlands_onehot.shape

(4202, 303)

In [19]:
#Here we are showing the mean of what each neighborhood has close by
netherlands_grouped = netherlands_onehot.groupby('City').mean().reset_index()
netherlands_grouped.head()

,City,Zoo,ATM,Accessories Store,Afghan Restaurant,Airport Lounge,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brasserie,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Camera Store,Campground,Candy Store,Caribbean Restaurant,Casino,Castle,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Distillery,Dog Run,Donut Shop,Drugstore,Dutch Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fireworks Store,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Friterie,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Field,Hockey Rink,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Hunting Supply,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Island,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Library,Light Rail Station,Lighthouse,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Notary,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Perfume Shop,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pool Hall,Pub,Public Art,Racetrack,Record Shop,Recreation Center,Rental Car Location,Rental Service,Resort,Rest Area,Restaurant,River,Road,Rock Club,Roof Deck,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shawarma Place,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Smoke Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Stadium,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Train Station,Tram Station,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Resta

In [20]:
netherlands_grouped.shape

(404, 303)

In [21]:
#Now, we are going to show what has more frequency in each of those neighborhoods.
num_top_venues = 5

for hood in netherlands_grouped['City']:
    print("----"+hood+"----")
    temp = netherlands_grouped[netherlands_grouped['City'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----Aalsmeer----
                  venue  freq
0            Restaurant  0.17
1           Supermarket  0.08
2                   Bar  0.08
3  Fast Food Restaurant  0.08
4  Gym / Fitness Center  0.08


----Aalten----
               venue  freq
0        Supermarket  0.25
1     Discount Store  0.12
2                Pub  0.12
3  German Restaurant  0.12
4   Department Store  0.12


----Abbenbroek----
          venue  freq
0   Coffee Shop  0.33
1          Café  0.33
2   Supermarket  0.33
3  Neighborhood  0.00
4  Optical Shop  0.00


----Alblasserdam----
           venue  freq
0    Supermarket  0.33
1           Pool  0.11
2  Grocery Store  0.11
3       Bus Stop  0.11
4  Shopping Mall  0.11


----Alkmaar----
                    venue  freq
0             Supermarket  0.12
1  Furniture / Home Store  0.12
2          Breakfast Spot  0.06
3              Sports Bar  0.06
4             Snack Place  0.06


----Almelo----
              venue  freq
0        Steakhouse   0.2
1     Train Station   0.2
2    

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [23]:
#And add it into a pandas dataframe to be able to apply afterwards Kclustering.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
cities_sorted = pd.DataFrame(columns=columns)
cities_sorted['City'] = netherlands_grouped['City']

for ind in np.arange(netherlands_grouped.shape[0]):
    cities_sorted.iloc[ind, 1:] = return_most_common_venues(netherlands_grouped.iloc[ind, :], num_top_venues)

cities_sorted.head()

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aalsmeer,Restaurant,Drugstore,Fish Market,Sporting Goods Shop,Fast Food Restaurant,Park,Bar,Falafel Restaurant,Gym / Fitness Center,Chinese Restaurant
1,Aalten,Supermarket,German Restaurant,Pub,Drugstore,Discount Store,Department Store,Bar,English Restaurant,Event Space,Factory
2,Abbenbroek,Supermarket,Café,Coffee Shop,Yoga Studio,English Restaurant,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market
3,Alblasserdam,Supermarket,Pizza Place,Grocery Store,Shopping Mall,Bus Stop,Snack Place,Pool,Creperie,Farm,English Restaurant
4,Alkmaar,Furniture / Home Store,Supermarket,Playground,Sporting Goods Shop,Electronics Store,Beach,Thrift / Vintage Store,Discount Store,Sports Bar,Gym / Fitness Center


In [24]:
# set number of clusters, in this case we choose 3 clusters because if its >3 we will have clusters with only one data in it.
kclusters = 4

netherlands_grouped_clustering = netherlands_grouped.drop('City', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(netherlands_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 3, 3, 3, 0, 3, 0, 0, 3, 0], dtype=int32)

In [25]:
Netherlands_Clean.head()

,City,Latitude,Longitude
0,The Hague,52.083333,4.300000
1,Amsterdam,52.350000,4.916667
2,Rotterdam,51.916667,4.500000
3,Utrecht,52.093813,5.119095
4,Eindhoven,51.450000,5.466667


In [26]:

# add clustering labels
cities_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

netherlands_merged = Netherlands_Clean

# merge netherlands_merged with cities_sorted to add latitude/longitude for each neighborhood
netherlands_merged = netherlands_merged.join(cities_sorted.set_index('City'), on='City')

netherlands_merged.head() # check the last columns!


,City,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Hague,52.083333,4.300000,0.0,Italian Restaurant,Restaurant,French Restaurant,Hotel,Coffee Shop,Bar,Wine Bar,Gym,Indian Restaurant,Supermarket
1,Amsterdam,52.350000,4.916667,0.0,Coffee Shop,Café,Restaurant,Platform,Bus Station,Breakfast Spot,Bus Stop,Sandwich Place,French Restaurant,Roof Deck
2,Rotterdam,51.916667,4.500000,0.0,Boat or Ferry,Bar,Breakfast Spot,Pool,Bistro,Island,Bus Stop,Field,Factory,Falafel Restaurant
3,Utrecht,52.093813,5.119095,0.0,Restaurant,Bar,Coffee Shop,Burger Joint,Vietnamese Restaurant,Plaza,Indonesian Restaurant,Gay Bar,Sandwich Place,Italian Restaurant
4,Eindhoven,51.450000,5.466667,0.0,Turkish Restaurant,Supermarket,Bakery,Kids Store,Fruit & Vegetable Store,Restaurant,Spanish Restaurant,Department Store,Grocery Store,Falafel Restaurant


In [27]:
cities_sorted.head()

,Cluster Labels,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Aalsmeer,Restaurant,Drugstore,Fish Market,Sporting Goods Shop,Fast Food Restaurant,Park,Bar,Falafel Restaurant,Gym / Fitness Center,Chinese Restaurant
1,3,Aalten,Supermarket,German Restaurant,Pub,Drugstore,Discount Store,Department Store,Bar,English Restaurant,Event Space,Factory
2,3,Abbenbroek,Supermarket,Café,Coffee Shop,Yoga Studio,English Restaurant,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market
3,3,Alblasserdam,Supermarket,Pizza Place,Grocery Store,Shopping Mall,Bus Stop,Snack Place,Pool,Creperie,Farm,English Restaurant
4,0,Alkmaar,Furniture / Home Store,Supermarket,Playground,Sporting Goods Shop,Electronics Store,Beach,Thrift / Vintage Store,Discount Store,Sports Bar,Gym / Fitness Center


In [28]:
#Deletint the NaN Cluster Labels and making them int for ploting purposes in the future.
cities_sorted.shape
cities_sorted = cities_sorted.dropna(axis='rows')
cities_sorted['Cluster Labels'] = cities_sorted['Cluster Labels'].astype(int)
cities_sorted.head()

,Cluster Labels,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Aalsmeer,Restaurant,Drugstore,Fish Market,Sporting Goods Shop,Fast Food Restaurant,Park,Bar,Falafel Restaurant,Gym / Fitness Center,Chinese Restaurant
1,3,Aalten,Supermarket,German Restaurant,Pub,Drugstore,Discount Store,Department Store,Bar,English Restaurant,Event Space,Factory
2,3,Abbenbroek,Supermarket,Café,Coffee Shop,Yoga Studio,English Restaurant,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market
3,3,Alblasserdam,Supermarket,Pizza Place,Grocery Store,Shopping Mall,Bus Stop,Snack Place,Pool,Creperie,Farm,English Restaurant
4,0,Alkmaar,Furniture / Home Store,Supermarket,Playground,Sporting Goods Shop,Electronics Store,Beach,Thrift / Vintage Store,Discount Store,Sports Bar,Gym / Fitness Center


In [29]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=7)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(netherlands_merged['Latitude'], netherlands_merged['Longitude'], netherlands_merged['City'], cities_sorted['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [30]:
#Cluster 0 (Brown color)
netherlands_merged.loc[netherlands_merged['Cluster Labels'] == 0, netherlands_merged.columns[[0] + list(range(4, netherlands_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Hague,Italian Restaurant,Restaurant,French Restaurant,Hotel,Coffee Shop,Bar,Wine Bar,Gym,Indian Restaurant,Supermarket
1,Amsterdam,Coffee Shop,Café,Restaurant,Platform,Bus Station,Breakfast Spot,Bus Stop,Sandwich Place,French Restaurant,Roof Deck
2,Rotterdam,Boat or Ferry,Bar,Breakfast Spot,Pool,Bistro,Island,Bus Stop,Field,Factory,Falafel Restaurant
3,Utrecht,Restaurant,Bar,Coffee Shop,Burger Joint,Vietnamese Restaurant,Plaza,Indonesian Restaurant,Gay Bar,Sandwich Place,Italian Restaurant
4,Eindhoven,Turkish Restaurant,Supermarket,Bakery,Kids Store,Fruit & Vegetable Store,Restaurant,Spanish Restaurant,Department Store,Grocery Store,Falafel Restaurant
5,Haarlemmerliede,Bar,Sporting Goods Shop,Monument / Landmark,Comedy Club,Farm,Yoga Studio,Fast Food Restaurant,Event Space,Factory,Falafel Restaurant
9,Leeuwarden,Furniture / Home Store,Snack Place,Fish & Chips Shop,Bus Stop,Music Venue,Yoga Studio,Event Space,Factory,Falafel Restaurant,Farm
10,Maastricht,Hotel,Italian Restaurant,Pub,French Restaurant,Coffee Shop,Supermarket,Asian Restaurant,Clothing Store,Theater,Shoe Store
12,Assen,Restaurant,Shopping Mall,Coffee Shop,Diner,Italian Restaurant,Supermarket,Bar,Ice Cream Shop,Bakery,Sandwich Place
13,Middelburg,Café,Bar,Restaurant,Plaza,Coffee Shop,Beer Garden,Furniture / Home Store,Bed & Breakfast,Italian Restaurant,French Restaurant


In [31]:
#Cluster 1 (Red color)
netherlands_merged.loc[netherlands_merged['Cluster Labels'] == 1, netherlands_merged.columns[[0] + list(range(4, netherlands_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
159,Oude Pekela,Bar,Bus Stop,Yoga Studio,Fast Food Restaurant,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Field
199,IJmuiden,Bus Stop,Middle Eastern Restaurant,History Museum,Volleyball Court,Paper / Office Supplies Store,Farmers Market,English Restaurant,Event Space,Factory,Falafel Restaurant
205,Montfoort,Bus Stop,Bookstore,Shoe Store,Tennis Court,Department Store,Supermarket,Event Space,Factory,Falafel Restaurant,Farm
246,Appingedam,Department Store,Bus Stop,Fast Food Restaurant,Field,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Yoga Studio
296,Nieuwerkerk aan den IJssel,Motorcycle Shop,Bus Stop,Yoga Studio,Fast Food Restaurant,English Restaurant,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market
308,Bergen,Bus Stop,Yoga Studio,Fast Food Restaurant,English Restaurant,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Field
316,Leusden,Bus Stop,Park,Yoga Studio,Farmers Market,English Restaurant,Event Space,Factory,Falafel Restaurant,Farm,Fast Food Restaurant
329,Rhenen,Bus Stop,Hockey Rink,Snack Place,Gym Pool,Farmers Market,English Restaurant,Event Space,Factory,Falafel Restaurant,Farm
360,Stadskanaal,Pool,Bus Stop,Farmers Market,Electronics Store,English Restaurant,Event Space,Factory,Falafel Restaurant,Farm,Fast Food Restaurant
378,Naaldwijk,Fast Food Restaurant,Flower Shop,Bus Stop,Park,Farmers Market,English Restaurant,Event Space,Factory,Falafel Restaurant,Farm


In [32]:
#Cluster 2 (Blue color)
netherlands_merged.loc[netherlands_merged['Cluster Labels'] == 2, netherlands_merged.columns[[0] + list(range(4, netherlands_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
127,Tilburg,Basketball Court,Yoga Studio,Fast Food Restaurant,English Restaurant,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Field
366,Son en Breugel,Basketball Court,Yoga Studio,Fast Food Restaurant,English Restaurant,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Field


In [33]:
#Cluster 3 (Violet color)
netherlands_merged.loc[netherlands_merged['Cluster Labels'] == 3, netherlands_merged.columns[[0] + list(range(4, netherlands_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Groningen,Supermarket,Pizza Place,Indian Restaurant,Fast Food Restaurant,Bus Stop,Beer Garden,Sandwich Place,Bar,Bakery,Indonesian Restaurant
7,Arnhem,Bus Stop,Supermarket,Pizza Place,Snack Place,Liquor Store,Restaurant,Fast Food Restaurant,Farm,English Restaurant,Event Space
8,’s-Hertogenbosch,Bus Stop,Electronics Store,Discount Store,Supermarket,Sports Club,Grocery Store,Gym / Fitness Center,Shoe Store,Plaza,Fast Food Restaurant
11,Zwolle,Hotel,Gym,Arcade,Coffee Shop,Supermarket,Yoga Studio,Farmers Market,English Restaurant,Event Space,Factory
17,Bergeijk,Supermarket,Bar,Snack Place,Department Store,Middle Eastern Restaurant,Drugstore,Restaurant,Chinese Restaurant,Diner,Shoe Store
19,Grootegast,Supermarket,Fast Food Restaurant,Electronics Store,English Restaurant,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Yoga Studio
27,Diever,Gastropub,Supermarket,Bar,Diner,Dessert Shop,English Restaurant,Factory,Falafel Restaurant,Farm,Farmers Market
28,Uithuizen,Plaza,Fast Food Restaurant,Discount Store,Train Station,Chinese Restaurant,Supermarket,Department Store,Electronics Store,Field,Fireworks Store
31,Zundert,Bar,Supermarket,Friterie,Restaurant,Gym,Shawarma Place,Clothing Store,Drugstore,Bus Station,Farm
32,Nunspeet,Bowling Alley,Supermarket,Snack Place,Pool,Electronics Store,English Restaurant,Event Space,Factory,Falafel Restaurant,Farm
